In [38]:
import numpy as np
import xml.etree.ElementTree as ET
import pytransform3d.visualizer as pv
from pytransform3d.urdf import UrdfTransformManager

# Make sure you have installed numpy, scipy, sympy, matplotlib, and pytransform3d:
# pip install numpy scipy sympy matplotlib pytransform3d[all]

# Documentation for pytransform3d: https://dfki-ric.github.io/pytransform3d/api.html

In [39]:
#%% ========== Load URDF ==========
tm = UrdfTransformManager()

URDF_Path = "Ufactory_xArm_7/Ufactory_xArm_7.urdf"
with open(URDF_Path, "r") as f:
    URDF_text = f.read()
    
tm.load_urdf(URDF_text, mesh_path="")

# --- Parse the URDF XML to get the robot name (value of <robot name="...">)
root = ET.fromstring(URDF_text)
robot_name = str(root.get("name"))

# --- Parse the URDF XML to get the robot joint names (excluding fixed joints)
joint_names = [str(j.get("name")) for j in root.findall(".//joint") if (j.get("type") or "").lower() not in ("fixed",)]
# print("Joint names (from URDF):", joint_names)

In [40]:
#%% ========== Set Joint Values ==========
n_DOF = len(joint_names)

# Desired joint angles (radians), for example zero/home configuration
theta_des = np.zeros(n_DOF)
for i, joint_name in enumerate(joint_names):
    tm.set_joint(joint_name, theta_des[i])

In [41]:
#%% ========== Robot Setup for Visualization ==========
fig = pv.figure(window_name='Animation', width=800*2, height=600*2)
graph = fig.plot_graph(tm, frame=robot_name, show_visuals=True, show_connections=False, show_frames=True, s=0.1)

# Example of ploting a frame
fig.plot_transform(np.eye(4), s=0.2)

fig.view_init(azim = 60, elev = 30) # Adjust view angle
fig.set_zoom(1.5) # Adjust zoom level
fig.show() # This will open an visualization window. Comment this to deactivate.

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


In [42]:
# %% ========== Define Desired Trajectory ==========
 # Example of a trivial desired trajectory: sinusoidal joint angles
dt = 0.01
t_f = 2
t = np.arange(0, t_f + dt, dt)
theta_des = np.zeros((len(t), n_DOF))
for i in range(len(t)):
    theta_des[i] = np.sin(np.pi * t[i] / t_f) * np.ones(n_DOF)


In [43]:
#%% ========== Animation Callback Function ==========
def animation_callback(step, tm, graph, joint_names, theta_des):
    for i, joint_name in enumerate(joint_names):
        tm.set_joint(joint_name, theta_des[step, i])
    graph.set_data()
    return graph

In [44]:
#%% ========== Robot Setup for Animation ==========
fig = pv.figure(window_name='Animation', width=800*2, height=600*2)
graph = fig.plot_graph(tm, frame=robot_name, show_visuals=True, show_connections=False, show_frames=True, s=0.1)

fig.view_init(azim = 60, elev = 30) # Adjust view angle
fig.set_zoom(1.5) # Adjust zoom level
n_frames = len(t)
fig.animate(animation_callback, n_frames, loop = False, fargs=(tm, graph, joint_names, theta_des))
fig.show()